In [34]:
import pandas as pd
import numpy as np
import optuna
import matplotlib.pyplot as plt
import networkx as nx
import itertools
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import SelectKBest, f_classif, SelectFromModel
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import os
import joblib

In [35]:
stage_3_samples = pd.read_csv('../data/cancer/stage_3_prostate_cancer_samples.csv')
stage_3_samples.shape

(150, 2570)

In [36]:
stage_4_samples = pd.read_csv('../data/cancer/stage_4_prostate_cancer_samples.csv')
stage_4_samples.shape

(91, 2570)

In [37]:
combined_dataset = pd.concat([stage_3_samples, stage_4_samples], ignore_index=True)

In [38]:
# Verify and clean the Stage column
print("Unique values in Stage column:", combined_dataset['Stage'].unique())
combined_dataset['Stage'] = combined_dataset['Stage'].str.strip()
combined_dataset['ID_REF'] = np.where(combined_dataset['Stage'] == 'Stage: 3', 0, 1)
combined_dataset['ID_REF'] = np.where(combined_dataset['Stage'] == 'Stage: 4', 1, combined_dataset['ID_REF'])

# Print class distribution to ensure both classes are present
print("Class distribution in ID_REF column:")
print(combined_dataset['ID_REF'].value_counts())


Unique values in Stage column: ['Stage: 3' 'Stage: 4']
Class distribution in ID_REF column:
ID_REF
0    150
1     91
Name: count, dtype: int64


In [39]:
# Define process_data function
def process_data(data, under_sample_factor=None, over_sample_factor=None):
    columns_to_drop = ['Sample_ID', 'Sex', 'Age', 'Stage', 'Disease']
    data = data.drop(columns=columns_to_drop, axis=1)
    
    x = np.array(data.drop(["ID_REF"], axis=1)).astype('float')
    y = np.array(data["ID_REF"]).astype('int')
    feature_names = data.columns[1:]

    if under_sample_factor is not None and isinstance(under_sample_factor, float) and 0 < under_sample_factor <= 1:
        under_sampler = RandomUnderSampler(sampling_strategy=under_sample_factor)
        x, y = under_sampler.fit_resample(x, y)

    if over_sample_factor is not None and isinstance(over_sample_factor, float) and 0 < over_sample_factor <= 1:
        over_sampler = RandomOverSampler(sampling_strategy=over_sample_factor)
        x, y = over_sampler.fit_resample(x, y)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0, stratify=y)

    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    return x_train, x_test, y_train, y_test, feature_names

In [40]:
# Define parameters
feature_selection_num = 50
feature_importance_num = 10

# Process data
x_train, x_test, y_train, y_test, feature_names = process_data(combined_dataset)

In [41]:
def svm_objective(trial):
    k = feature_selection_num
    
    C = trial.suggest_float('C', 1e-3, 1e3, log=True)
    kernel = 'linear'  # Use only linear kernel for feature extraction
    
    # Relaxed LassoCV with a wider range of smaller alphas
    lasso = SelectFromModel(LassoCV(alphas=np.logspace(-6, -1, 50), cv=5, max_iter=10000, random_state=0))
    
    pipe = Pipeline([
        ('skb', SelectKBest(f_classif, k=k)),
        ('lasso', lasso),
        ('estimator', SVC(C=C, kernel=kernel, random_state=0))
    ])
    
    cv = StratifiedKFold(n_splits=5)
    try:
        scores = cross_val_score(pipe, x_train, y_train, cv=cv, scoring='accuracy')
        return scores.mean()
    except ValueError as e:
        print(f"ValueError: {e}")
        print("Selected features by SelectKBest:", skb.get_support(indices=True))
        if hasattr(lasso.estimator_, 'coef_'):
            print("Lasso coefficients:", lasso.estimator_.coef_)
        return np.nan

In [42]:
def rf_objective(trial):
    k = feature_selection_num
    
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    
    lasso = SelectFromModel(LassoCV(alphas=np.logspace(-6, -1, 50), cv=5, max_iter=10000, random_state=0))
    
    pipe = Pipeline([
        ('skb', SelectKBest(f_classif, k=k)),
        ('lasso', lasso),
        ('estimator', RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features, criterion=criterion, random_state=0))
    ])
    
    cv = StratifiedKFold(n_splits=5)
    scores = cross_val_score(pipe, x_train, y_train, cv=cv, scoring='accuracy')
    return scores.mean()

In [43]:
def xgboost_objective(trial):
    k = feature_selection_num
    
    learning_rate = trial.suggest_float('learning_rate', 1e-3, 1.0, log=True)
    max_depth = trial.suggest_int('max_depth', 2, 10)
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    
    lasso = SelectFromModel(LassoCV(alphas=np.logspace(-6, -1, 50), cv=5, max_iter=10000, random_state=0))
    
    pipe = Pipeline([
        ('skb', SelectKBest(f_classif, k=k)),
        ('lasso', lasso),
        ('estimator', XGBClassifier(learning_rate=learning_rate, max_depth=max_depth, n_estimators=n_estimators, random_state=0, use_label_encoder=False, eval_metric='logloss'))
    ])
    
    cv = StratifiedKFold(n_splits=5)
    scores = cross_val_score(pipe, x_train, y_train, cv=cv, scoring='accuracy')
    return scores.mean()

In [44]:
# Function to save study
def save_study(study, filename):
    joblib.dump(study, filename)

# Function to load study
def load_study(filename):
    return joblib.load(filename)

In [45]:
# Optimize hyperparameters using Optuna
svm_study_filename = '50_svm_study_s3_s4.pkl'
rf_study_filename = '50_rf_study_s3_s4.pkl'
xgboost_study_filename = '50_xgboost_study_s3_s4.pkl'

In [46]:
# Optimize hyperparameters using Optuna with early stopping
def optimize_with_early_stopping(objective, study_filename, n_trials=50, patience=10):
    if os.path.exists(study_filename):
        study = load_study(study_filename)
        return study
    else:
        study = optuna.create_study(direction='maximize')
    
    best_value = -np.inf
    trials_without_improvement = 0
    
    for trial in range(n_trials):
        study.optimize(objective, n_trials=1)
        
        current_best_value = study.best_value
        if current_best_value > best_value:
            best_value = current_best_value
            trials_without_improvement = 0
        else:
            trials_without_improvement += 1
        
        if trials_without_improvement >= patience:
            print(f"Early stopping at trial {trial + 1}")
            break
        
        save_study(study, study_filename)
    
    return study

### Find Hyperparmeters if not trained already

In [47]:
svm_study = optimize_with_early_stopping(svm_objective, svm_study_filename, n_trials=50, patience=10)

In [48]:
rf_study = optimize_with_early_stopping(rf_objective, rf_study_filename, n_trials=50, patience=10)

In [49]:
xgboost_study = optimize_with_early_stopping(xgboost_objective, xgboost_study_filename, n_trials=50, patience=10)

### Print the best trial for each study

For SVM, RF, and XGBoost

In [50]:
print("Best SVM trial:")
svm_trial = svm_study.best_trial
print("  Value: ", svm_trial.value)
print("  Params: ")
for key, value in svm_trial.params.items():
    print(f"    {key}: {value}")

Best SVM trial:
  Value:  0.6251012145748988
  Params: 
    C: 0.0016017877344232227


In [51]:
print("Best Random Forest trial:")
rf_trial = rf_study.best_trial
print("  Value: ", rf_trial.value)
print("  Params: ")
for key, value in rf_trial.params.items():
    print(f"    {key}: {value}")

Best Random Forest trial:
  Value:  0.5882591093117409
  Params: 
    n_estimators: 961
    max_depth: 2
    max_features: sqrt
    criterion: entropy


In [52]:
print("Best XGBoost trial:")
xgboost_trial = xgboost_study.best_trial
print("  Value: ", xgboost_trial.value)
print("  Params: ")
for key, value in xgboost_trial.params.items():
    print(f"    {key}: {value}")

Best XGBoost trial:
  Value:  0.6251012145748988
  Params: 
    learning_rate: 0.0022415553355806675
    max_depth: 3
    n_estimators: 113


## Train and evaluate the models with the best hyperparameters

In [53]:
# Train and evaluate the models with the best hyperparameters
def train_and_evaluate(pipe, x_train, y_train, x_test, y_test):
    pipe.fit(x_train, y_train)
    y_pred = pipe.predict(x_test)
    print(f'Testing accuracy {accuracy_score(y_test, y_pred)}')
    print(f'Confusion matrix: \n{confusion_matrix(y_test, y_pred)}')

### SVM

In [54]:
best_svm_params = svm_trial.params
svm_pipe = Pipeline([
    ('skb', SelectKBest(f_classif, k=feature_selection_num)),
    ('lasso', SelectFromModel(LassoCV(alphas=np.logspace(-6, -1, 50), cv=5, max_iter=10000, random_state=0))),
    ('estimator', SVC(C=best_svm_params['C'], kernel='linear', random_state=0))  # Ensure linear kernel
])
train_and_evaluate(svm_pipe, x_train, y_train, x_test, y_test)

Testing accuracy 0.6122448979591837
Confusion matrix: 
[[30  0]
 [19  0]]


In [55]:
# svm_accuracies = []

# # Loop through the number of features from 1 to 150
# for k in range(1, 121):
#     best_svm_params = svm_trial.params
#     svm_pipe = Pipeline([
#         ('skb', SelectKBest(f_classif, k=k)),
#         ('lasso', SelectFromModel(LassoCV(cv=5, random_state=0))),
#         ('estimator', SVC(C=best_svm_params['C'], kernel=best_svm_params['kernel'], random_state=0))
#     ])
    
#     # Train and evaluate the pipeline
#     svm_pipe.fit(x_train, y_train)
#     y_pred = svm_pipe.predict(x_test)
#     accuracy = accuracy_score(y_test, y_pred)
#     svm_accuracies.append(accuracy)
#     print(f'Number of features: {k}, Accuracy: {accuracy}')

# # Plot the number of features vs accuracy
# plt.figure(figsize=(10, 6))
# plt.plot(range(1, 121), svm_accuracies, marker='o')
# plt.title('Accuracy vs Number of Features (SVM)')
# plt.xlabel('Number of Features')
# plt.ylabel('Accuracy')
# plt.grid(True)
# plt.show()

### Random Forest

In [56]:
best_rf_params = rf_trial.params
rf_pipe = Pipeline([
    ('skb', SelectKBest(f_classif, k=feature_selection_num)),
    ('lasso', SelectFromModel(LassoCV(alphas=np.logspace(-6, -1, 50), cv=5, max_iter=10000, random_state=0))),
    ('estimator', RandomForestClassifier(n_estimators=best_rf_params['n_estimators'],
                                         max_depth=best_rf_params['max_depth'],
                                         max_features=best_rf_params['max_features'],
                                         criterion=best_rf_params['criterion'],
                                         random_state=0))
])
train_and_evaluate(rf_pipe, x_train, y_train, x_test, y_test)

Testing accuracy 0.6122448979591837
Confusion matrix: 
[[26  4]
 [15  4]]


In [57]:
# rf_accuracies = []

# # Loop through the number of features from 1 to 200
# for k in range(1, 121):
#     best_rf_params = rf_trial.params
#     rf_pipe = Pipeline([
#         ('skb', SelectKBest(f_classif, k=k)),
#         ('lasso', SelectFromModel(LassoCV(cv=5, random_state=0))),
#         ('estimator', RandomForestClassifier(n_estimators=best_rf_params['n_estimators'],
#                                              max_depth=best_rf_params['max_depth'],
#                                              max_features=best_rf_params['max_features'],
#                                              criterion=best_rf_params['criterion'],
#                                              random_state=0))
#     ])
    
#     # Train and evaluate the pipeline
#     rf_pipe.fit(x_train, y_train)
#     y_pred = rf_pipe.predict(x_test)
#     rf_accuracy = accuracy_score(y_test, y_pred)
#     rf_accuracies.append(rf_accuracy)
#     print(f'Number of features: {k}, Accuracy: {rf_accuracy}')

# # Plot the number of features vs rf_accuracy
# plt.figure(figsize=(10, 6))
# plt.plot(range(1, 121), rf_accuracies, marker='o')
# plt.title('Accuracy vs Number of Features')
# plt.xlabel('Number of Features')
# plt.ylabel('Accuracy')
# plt.grid(True)
# plt.show()

### XGBoost

In [58]:
best_xgboost_params = xgboost_trial.params
xgboost_pipe = Pipeline([
    ('skb', SelectKBest(f_classif, k=feature_selection_num)),
    ('lasso', SelectFromModel(LassoCV(alphas=np.logspace(-6, -1, 50), cv=5, max_iter=10000, random_state=0))),
    ('estimator', XGBClassifier(learning_rate=best_xgboost_params['learning_rate'],
                                max_depth=best_xgboost_params['max_depth'],
                                n_estimators=best_xgboost_params['n_estimators'],
                                random_state=0,
                                use_label_encoder=False,
                                eval_metric='logloss'))
])
train_and_evaluate(xgboost_pipe, x_train, y_train, x_test, y_test)

Testing accuracy 0.6122448979591837
Confusion matrix: 
[[30  0]
 [19  0]]


In [59]:
# xgboost_accuracies = []

# # Loop through the number of features from 1 to 150
# for k in range(1, 121):
#     best_xgboost_params = xgboost_trial.params
#     xgboost_pipe = Pipeline([
#         ('skb', SelectKBest(f_classif, k=k)),
#         ('lasso', SelectFromModel(LassoCV(cv=5, random_state=0))),
#         ('estimator', XGBClassifier(learning_rate=best_xgboost_params['learning_rate'],
#                                     max_depth=best_xgboost_params['max_depth'],
#                                     n_estimators=best_xgboost_params['n_estimators'],
#                                     random_state=0,
#                                     use_label_encoder=False,
#                                     eval_metric='logloss'))
#     ])

#     # Train and evaluate the pipeline
#     xgboost_pipe.fit(x_train, y_train)
#     y_pred = xgboost_pipe.predict(x_test)
#     xgboost_accuracy = accuracy_score(y_test, y_pred)
#     xgboost_accuracies.append(xgboost_accuracy)
#     print(f'Number of features: {k}, Accuracy: {xgboost_accuracy}') 

# # Plot the number of features vs xgboost_accuracy
# plt.figure(figsize=(10, 6))
# plt.plot(range(1, 121), xgboost_accuracies, marker='o')
# plt.title('Accuracy vs Number of Features (XGBoost)')
# plt.xlabel('Number of Features')
# plt.ylabel('Accuracy')
# plt.grid(True)
# plt.show()

In [60]:
def get_top_features(pipe, feature_names, top_feature_num):
    if isinstance(pipe.named_steps['estimator'], SVC):
        if pipe.named_steps['estimator'].kernel != 'linear':
            raise ValueError("Feature importance is not available for non-linear SVM kernels.")
        feature_scores = pipe.named_steps['estimator'].coef_[0]
    elif isinstance(pipe.named_steps['estimator'], XGBClassifier):
        feature_scores = pipe.named_steps['estimator'].feature_importances_
    else:
        raise ValueError("Unsupported estimator type for feature extraction.")
    
    # Get selected features from SelectKBest
    skb_support = pipe.named_steps['skb'].get_support(indices=True)
    print("Selected features from SelectKBest:", skb_support)
    
    # Transform the features using SelectKBest
    skb_features = pipe.named_steps['skb'].transform(x_train)
    
    # Get selected features from LassoCV
    lasso = LassoCV(alphas=np.logspace(-6, -1, 50), cv=5, max_iter=10000, random_state=0)
    lasso.fit(skb_features, y_train)
    lasso_support = np.where(lasso.coef_ != 0)[0]
    print("Selected features from LassoCV:", lasso_support)
    
    if len(lasso_support) == 0:
        print("No features selected after LassoCV.")
        return []
    
    # Ensure indices match feature_scores
    top_indices = np.argsort(np.abs(lasso.coef_[lasso_support]))[::-1][:top_feature_num]
    top_features = [(feature_names[skb_support[i]], lasso.coef_[lasso_support[i]]) for i in top_indices]
    
    return top_features

In [61]:
try:
    svm_top_features = get_top_features(svm_pipe, feature_names, feature_importance_num)
    print("Top SVM features:", svm_top_features)
except ValueError as e:
    print("SVM feature extraction error:", e)

xgboost_top_features = get_top_features(xgboost_pipe, feature_names, feature_importance_num)
print("Top XGBoost features:", xgboost_top_features)

Selected features from SelectKBest: [ 153  171  176  177  180  183  184  187  191  200  205  213  215  223
  239  275  311  418  428  444  457  585  834  845  852  859  920  933
  946  953  981  988  991 1132 1271 1290 1320 1348 1401 1411 1424 1446
 1454 1460 1491 1501 1537 1655 2058 2436]
Selected features from LassoCV: [ 1  3  7  9 12 14 16 20 21 23 24 28 32 33 34 37 38 39 40 41 43 44 45 47
 48 49]
Top SVM features: [('hsa-miR-4474-3p', -0.10608082221855654), ('hsa-miR-4511', 0.091886225558603), ('hsa-miR-6769b-5p', -0.06581028808849043), ('hsa-miR-1250-5p', 0.058384490389818), ('hsa-miR-520d-3p', -0.05240977497784932), ('hsa-miR-6715b-5p', 0.046224337586538135), ('hsa-miR-3074-3p', -0.042191496624585684), ('hsa-miR-4646-5p', -0.0390558359936401), ('hsa-miR-892c-3p', 0.035575915306801584), ('hsa-miR-3201', 0.03397184700007616)]
Selected features from SelectKBest: [ 153  171  176  177  180  183  184  187  191  200  205  213  215  223
  239  275  311  418  428  444  457  585  834  845 

In [62]:
def normalize_importance(importance_scores):
    max_score = max(importance_scores, key=lambda x: abs(x[1]))[1]
    min_score = min(importance_scores, key=lambda x: abs(x[1]))[1]
    normalized_scores = [(feature, (score - min_score) / (max_score - min_score)) for feature, score in importance_scores]
    return normalized_scores

In [63]:
# Normalize the top features for SVM and XGBoost
normalized_svm_features = normalize_importance(svm_top_features)
normalized_xgboost_features = normalize_importance(xgboost_top_features)

# Compile normalized top features
def compile_normalized_top_features(normalized_svm_features, normalized_xgboost_features):
    top_features = []
    for feature, score in normalized_svm_features:
        top_features.append((f'SVM_{feature}', score))
    for feature, score in normalized_xgboost_features:
        top_features.append((f'XGB_{feature}', score))
    return top_features

normalized_top_features = compile_normalized_top_features(normalized_svm_features, normalized_xgboost_features)
print("Compiled normalized top features:", normalized_top_features)

Compiled normalized top features: [('SVM_hsa-miR-4474-3p', 1.0), ('SVM_hsa-miR-4511', -0.4135185632779206), ('SVM_hsa-miR-6769b-5p', 0.7124615021281688), ('SVM_hsa-miR-1250-5p', -0.1743104471049522), ('SVM_hsa-miR-520d-3p', 0.6167795477219882), ('SVM_hsa-miR-6715b-5p', -0.08748487733093413), ('SVM_hsa-miR-3074-3p', 0.5438192934814057), ('SVM_hsa-miR-4646-5p', 0.5214301405403033), ('SVM_hsa-miR-892c-3p', -0.011453321922921386), ('SVM_hsa-miR-3201', -0.0), ('XGB_hsa-miR-4474-3p', 1.0), ('XGB_hsa-miR-4511', -0.4135185632779206), ('XGB_hsa-miR-6769b-5p', 0.7124615021281688), ('XGB_hsa-miR-1250-5p', -0.1743104471049522), ('XGB_hsa-miR-520d-3p', 0.6167795477219882), ('XGB_hsa-miR-6715b-5p', -0.08748487733093413), ('XGB_hsa-miR-3074-3p', 0.5438192934814057), ('XGB_hsa-miR-4646-5p', 0.5214301405403033), ('XGB_hsa-miR-892c-3p', -0.011453321922921386), ('XGB_hsa-miR-3201', -0.0)]


In [64]:
normalized_top_features_df = pd.DataFrame(normalized_top_features, columns=['Feature', 'Normalized Importance'])
normalized_top_features_df.to_csv('../GSEA/miRNA/s3_s4_miRNA.csv', index=False)
print("Compiled normalized top features saved to '../GSEA/miRNA/s3_s4_miRNA.csv'")

Compiled normalized top features saved to '../GSEA/miRNA/s3_s4_miRNA.csv'


In [65]:
def get_all_selected_features(pipe, feature_names, classifier_name):
    if isinstance(pipe.named_steps['estimator'], SVC):
        if pipe.named_steps['estimator'].kernel != 'linear':
            raise ValueError("Feature importance is not available for non-linear SVM kernels.")
        feature_scores = pipe.named_steps['estimator'].coef_[0]
    elif isinstance(pipe.named_steps['estimator'], XGBClassifier):
        feature_scores = pipe.named_steps['estimator'].feature_importances_
    else:
        raise ValueError("Unsupported estimator type for feature extraction.")
    
    # Get selected features from SelectKBest
    skb_support = pipe.named_steps['skb'].get_support(indices=True)
    print("Selected features from SelectKBest:", skb_support)
    
    # Transform the features using SelectKBest
    skb_features = pipe.named_steps['skb'].transform(x_train)
    
    # Get selected features from LassoCV
    lasso = LassoCV(alphas=np.logspace(-6, -1, 50), cv=5, max_iter=10000, random_state=0)
    lasso.fit(skb_features, y_train)
    lasso_support = np.where(lasso.coef_ != 0)[0]
    print("Selected features from LassoCV:", lasso_support)
    
    # Map the selected LassoCV features back to the original feature names
    lasso_support_mapped = skb_support[lasso_support]
    
    # Ensure the index does not go out of bounds
    all_features = [(feature_names[i], feature_scores[j], classifier_name) for j, i in enumerate(lasso_support_mapped) if j < len(feature_scores)]
    
    return all_features

In [66]:
# Extract all selected features from SVM and XGBoost
all_svm_features = get_all_selected_features(svm_pipe, feature_names, 'SVM')
print("All SVM features:", all_svm_features)

all_xgboost_features = get_all_selected_features(xgboost_pipe, feature_names, 'XGBoost')
print("All XGBoost features:", all_xgboost_features)

# Combine the features and save to CSV
all_features_df = pd.DataFrame(all_svm_features + all_xgboost_features, columns=['Feature', 'Importance', 'Classifier'])
all_features_df.to_csv('../GSEA/miRNA/50_s3_s4.csv', index=False)
print("All selected features saved to '../GSEA/miRNA/50_s3_s4.csv'")

Selected features from SelectKBest: [ 153  171  176  177  180  183  184  187  191  200  205  213  215  223
  239  275  311  418  428  444  457  585  834  845  852  859  920  933
  946  953  981  988  991 1132 1271 1290 1320 1348 1401 1411 1424 1446
 1454 1460 1491 1501 1537 1655 2058 2436]
Selected features from LassoCV: [ 1  3  7  9 12 14 16 20 21 23 24 28 32 33 34 37 38 39 40 41 43 44 45 47
 48 49]
All SVM features: [('hsa-miR-1205', 0.014709901002911443, 'SVM'), ('hsa-miR-29b-1-5p', 0.02792880142555269, 'SVM'), ('hsa-miR-1250-5p', 0.02669011720572015, 'SVM'), ('hsa-miR-4516', 0.029116084772471396, 'SVM'), ('hsa-miR-6715b-5p', 0.028757845625186143, 'SVM'), ('hsa-miR-520d-3p', 0.01740820897502871, 'SVM'), ('hsa-miR-6769b-5p', -0.02576088186919639, 'SVM'), ('hsa-miR-3975', 0.0351761704282797, 'SVM'), ('hsa-miR-548h-5p', -0.03119944007537516, 'SVM'), ('hsa-miR-4474-3p', 0.01947720521751981, 'SVM'), ('hsa-miR-4511', 0.025615313848852257, 'SVM'), ('hsa-miR-6756-5p', 0.026932390329150988, 